# MIP after rotating in 3D - full example

In [1]:
from dask_image.ndinterp import affine_transform
import dask.array as da
from dask.diagnostics import ProgressBar
import zarr
import skimage

In [2]:
from scipy.spatial.transform import Rotation as R
import numpy as np
from scipy.linalg  import inv


def get_rotation_on_center(shape, degrees, axis_name):
        
    r0 = np.eye(4) 
    r0[0,3] = -shape[0]/2
    r0[1,3] = -shape[1]/2
    r0[2,3] = -shape[2]/2
    
    r1 = np.eye(4)
    r1[:3,:3]=R.from_euler(axis_name, degrees, degrees=True).as_matrix()

    #to get rotation around center of image: move origin, rotate, move origin back 
    return inv(r0) @ r1 @ r0

In [3]:
from pathlib import Path
import zarr
from tqdm import tqdm

subject='mouse1'
acq='4x'
level=3
mip_axis=0 
rot_axis='y'
channel=0
angle_increment=5
z_padding=0.4 #to avoid going out of fov with rotation

in_zarr_pattern = '/cifs/prado/Kelly/data/lightsheet/bids/test_bids/sub-{subject}/micr/sub-{subject}_sample-brain_acq-{acq}_spim.ome.zarr.zip'
in_zarr = in_zarr_pattern.format(subject=subject, acq=acq)

In [4]:
z_in = zarr.open(in_zarr)
attrs = z_in['/'].attrs.asdict()

#get voxel dimensions for the given level
coord_transforms = attrs['multiscales'][0]['datasets'][level]['coordinateTransformations']
coord_transforms
zscale=coord_transforms[0]['scale'][-3] 
xyscale=coord_transforms[0]['scale'][-2]

#zfactor is how much thicker (greater than 1) /thinner (less than 1) Z is than x and y dims
# ie we need to rescale z dim by this factor to get isotropic volume 
zfactor = zscale/xyscale
zfactor

zstep = int(np.round(1/zfactor))
zstep

5

In [5]:
darr_ds = da.from_zarr(in_zarr,component=f'/{level}')[channel][::zstep,:,:]
darr_pad = da.zeros((int(darr_ds.shape[0]*z_padding),darr_ds.shape[1],darr_ds.shape[2]),dtype='uint16')
darr_ds = da.concatenate([darr_pad,darr_ds,darr_pad],axis=0)
                    
out_dir=Path(f'sub-{subject}_acq-{acq}_channel-{channel}_level-{level}_rotate-{rot_axis}_axis-{mip_axis}_MIP')

#make out dir
out_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
#generate mips while rotating to make animation

for angle in tqdm(range(0,360,angle_increment)):
    darr_mip=affine_transform(darr_ds,get_rotation_on_center(darr_ds.shape,angle,rot_axis)).max(axis=mip_axis)
    out_file = out_dir / f'MIP_rotate-{angle:03d}.tif' 
    skimage.io.imsave(out_file,darr_mip)

  8%|███████▊                                                                                      | 6/72 [09:46<1:49:06, 99.19s/it]